In [1]:
import pandas as pd 
import numpy as np 

In [2]:
benef = pd.read_csv('../cleaned_data/benef_final.csv')

In [135]:
benef['Age'] = benef['Age']//365

In [137]:
X= benef.drop(['Fraud'], axis=1)
Y= benef['Fraud']

## Feature Selection

In [41]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs = -1, max_depth = 8,
                           min_samples_split= 0.3)
boruta = BorutaPy(estimator = rf, verbose=2, perc=90,
                  n_estimators = 'auto',
                  max_iter = 100)

In [46]:
boruta.fit(np.array(X), np.array(Y))

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	20
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	20
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	20
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	20
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	20
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	20
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	20
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	11
Tentative: 	6
Rejected: 	3
Iteration: 	9 / 100
Confirmed: 	11
Tentative: 	6
Rejected: 	3
Iteration: 	10 / 100
Confirmed: 	11
Tentative: 	6
Rejected: 	3
Iteration: 	11 / 100
Confirmed: 	11
Tentative: 	6
Rejected: 	3
Iteration: 	12 / 100
Confirmed: 	11
Tentative: 	6
Rejected: 	3
Iteration: 	13 / 100
Confirmed: 	11
Tentative: 	6
Rejected: 	3
Iteration: 	14 / 100
Confirmed: 	11
Tentative: 	6
Rejected: 	3
Iteration: 	15 / 100
Confirmed: 	11
Tentative: 	6
Rejected: 	3
Iteration: 	16 / 100
Confirmed: 	13
Tentative: 	4
Rejected: 	3
I

BorutaPy(estimator=RandomForestClassifier(max_depth=8, min_samples_split=0.3,
                                          n_estimators=68, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x118CE1640),
         n_estimators='auto', perc=90,
         random_state=RandomState(MT19937) at 0x118CE1640, verbose=2)

In [47]:
X.columns[boruta.support_].to_list()

['Is_inpatient',
 'Age',
 'Race',
 'State',
 'County',
 'Chronic_Alzheimer',
 'Chronic_Heartfailure',
 'Chronic_KidneyDisease',
 'Chronic_ObstrPulmonary',
 'Chronic_IschemicHeart',
 'Chronic_rheumatoidarthritis',
 'Chronic_stroke',
 'RenalDisease']

In [48]:
X.columns[boruta.support_weak_].to_list()

['FullYearPlanA']

In [50]:
set(X.columns.to_list())^set(X.columns[boruta.support_].to_list())

{'Chronic_Cancer',
 'Chronic_Depression',
 'Chronic_Diabetes',
 'Chronic_Osteoporasis',
 'FullYearPlanA',
 'FullYearPlanB',
 'Gender'}

## Decision Tree

In [51]:
features = X.columns[boruta.support_].to_list()

In [5]:
from sklearn import tree

In [138]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X[features],Y,
                                                    test_size=0.3, random_state=42)

In [139]:
clf = tree.DecisionTreeClassifier(max_depth=15, 
                                min_samples_split=0.25,
                                max_features='auto'
                                 )
clf = clf.fit(X_train, y_train)

In [140]:
y_pred = clf.predict(X_test)

In [141]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[96531  7199]
 [55516  8218]]
              precision    recall  f1-score   support

           0       0.63      0.93      0.75    103730
           1       0.53      0.13      0.21     63734

    accuracy                           0.63    167464
   macro avg       0.58      0.53      0.48    167464
weighted avg       0.60      0.63      0.55    167464



In [142]:
y_test.sum()

63734

In [143]:
import graphviz

In [144]:
dot_data = tree.export_graphviz(clf, out_file=None, 
                                feature_names=features,  
                                filled=True)

In [145]:
graph = graphviz.Source(dot_data, format="png")

In [146]:
print(tree.export_text(clf, feature_names=features))

|--- Chronic_KidneyDisease <= 0.50
|   |--- Race <= 2.50
|   |   |--- County <= 105.00
|   |   |   |--- class: 0
|   |   |--- County >  105.00
|   |   |   |--- State <= 10.50
|   |   |   |   |--- class: 1
|   |   |   |--- State >  10.50
|   |   |   |   |--- Is_inpatient <= 0.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- Is_inpatient >  0.50
|   |   |   |   |   |--- class: 1
|   |--- Race >  2.50
|   |   |--- class: 0
|--- Chronic_KidneyDisease >  0.50
|   |--- State <= 10.50
|   |   |--- class: 0
|   |--- State >  10.50
|   |   |--- Race <= 2.50
|   |   |   |--- County <= 0.50
|   |   |   |   |--- class: 0
|   |   |   |--- County >  0.50
|   |   |   |   |--- Chronic_rheumatoidarthritis <= 0.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- Chronic_rheumatoidarthritis >  0.50
|   |   |   |   |   |--- State <= 21.50
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- State >  21.50
|   |   |   |   |   |   |--- class: 0
|   |   |--- Race >  2.50
|   |   |   